<a href="https://colab.research.google.com/github/rulas99/thermal_anomaly_detection/blob/main/thermal_anomalies_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Semantic Segmentation [source](https://towardsdatascience.com/train-neural-net-for-semantic-segmentation-with-pytorch-in-50-lines-of-code-830c71a6544f)

In [1]:
import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf

from pandas import read_csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
Learning_Rate = 1e-5
width = 500
height = 500 # image width and height
batchSize = 7

In [4]:
modelPath = "/content/drive/MyDrive/Inge/segment_sentinel2_hotspot/Models_2/model_2030_v1.torch" 

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True) # Load net

Net.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 2 classes

Net = Net.to(device) # Set net to GPU or CPU

# Net.load_state_dict(torch.load(modelPath)) # Load trained model

optimizer = torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) # Create adam optimizer

weights = torch.FloatTensor([0.01,1]).cuda()
criterion = torch.nn.CrossEntropyLoss(weight = weights) # Set loss function

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

In [5]:
train1 = read_csv("/content/drive/MyDrive/segment_sentinel2_hotspot/train_2.csv")
train = read_csv("/content/drive/MyDrive/segment_sentinel2_hotspot/train_raw.csv")

In [6]:
#train2[~train2.label_path.isin(train1.label_path)]

In [7]:
transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)),tf.ToTensor(),])
                         #tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
                         
transformAnn=tf.Compose([tf.ToPILImage(),tf.Resize((height,width),tf.InterpolationMode.NEAREST),tf.ToTensor()])

In [8]:
def ReadRandomImage(): # First lets load random image and  the corresponding annotation
    idx=np.random.randint(0,len(train)) # Select random image
    thermal = cv2.imread(train.ori_path.iloc[idx])
    masked =  cv2.imread(train.label_path.iloc[idx],0)
    AnnMap = np.zeros(thermal.shape[0:2],np.float32)
    if masked is not None:  AnnMap[ masked == 255 ] = 1
    Img=transformImg(thermal)
    AnnMap=transformAnn(AnnMap)
    return Img,AnnMap


def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize,3,height,width])
    ann = torch.zeros([batchSize, height, width])
    for i in range(batchSize):
        images[i],ann[i]=ReadRandomImage()
    return images, ann

In [9]:
for itr in range(5000): # Training loop
   images,ann=LoadBatch() # Load taining batch
   images=torch.autograd.Variable(images,requires_grad=False).to(device) # Load image
   ann = torch.autograd.Variable(ann, requires_grad=False).to(device) # Load annotation
   Pred=Net(images)['out'] # make prediction
   optimizer.zero_grad()
   Loss=criterion(Pred,ann.long()) # Calculate cross entropy loss
   Loss.backward() # Backpropagate loss
   optimizer.step() # Apply gradient descent change to weight

   loss = round(float(Loss.data.cpu()),3)

   if itr % 250 == 0 or loss<0.06: #Save model weight once every 100 steps permenant file
        #accG = [(torch.argmax(Pred[i], 0).cpu().detach().numpy() == ann[i].cpu().detach().numpy()).sum()/(width*height) for i in range(batchSize)]
        #accM = round(sum(accG)/batchSize,5)
        print(itr,f") Loss = {loss}")
        #print(f"Saving model_{itr}.torch")
        torch.save(Net.state_dict(),   f'/content/drive/MyDrive/segment_sentinel2_hotspot/Models_2/model_{itr}_v3.torch')

        if loss<0.06:
          break

0 ) Loss = 0.597
250 ) Loss = 0.249
500 ) Loss = 0.191
750 ) Loss = 0.15
1000 ) Loss = 0.125
1250 ) Loss = 0.086
1500 ) Loss = 0.085
1750 ) Loss = 0.084
1878 ) Loss = 0.056
